# SEER Project
enter description here

Data Wrangling

Descriptive Analysis

Statistical Analysis

Machine Learning Model

## Data Wrangling

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sc.setLogLevel("ERROR")

In [4]:
#import necessary libraries
from pyspark.sql import functions as F
import pyspark.pandas as ps
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd


/opt/anaconda3/envs/bmi6318/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [5]:
#import data

seer_data = spark.read.csv('./SEER 2018-2022.txt', header = False, sep = '\t')

#### Acquire and Add Column Names to Dataframe

In [6]:
#text-processing for column names

dictionary_text = '''
Var1Name=Year of diagnosis
Var1DisplayType=Formatted
Var2Name=Age recode with <1 year olds and 90+
Var2DisplayType=Formatted
Var3Name=Sex
Var3DisplayType=Formatted
Var4Name=Race recode (W, B, AI, API)
Var4DisplayType=Formatted
Var5Name=Origin recode NHIA (Hispanic, Non-Hisp)
Var5DisplayType=Formatted
Var6Name=Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)
Var6DisplayType=Formatted
Var7Name=Median household income inflation adj to 2023
Var7DisplayType=Formatted
Var8Name=Rural-Urban Continuum Code
Var8DisplayType=Formatted
Var9Name=Sequence number
Var9DisplayType=Formatted
Var10Name=First malignant primary indicator
Var10DisplayType=Formatted
Var11Name=Primary by international rules
Var11DisplayType=Formatted
Var12Name=Record number recode
Var12DisplayType=Formatted
Var13Name=Total number of in situ/malignant tumors for patient
Var13DisplayType=Formatted
Var14Name=Total number of benign/borderline tumors for patient
Var14DisplayType=Formatted
Var15Name=AYA site recode 2020 Revision
Var15DisplayType=Formatted
Var16Name=SEER Brain and CNS Recode
Var16DisplayType=Formatted
Var17Name=Behavior code ICD-O-3
Var17DisplayType=Formatted
Var18Name=Primary Site - labeled
Var18DisplayType=Formatted
Var19Name=Primary Site
Var19DisplayType=Formatted
Var20Name=Histologic Type ICD-O-3
Var20DisplayType=Formatted
Var21Name=Combined Summary Stage with Expanded Regional Codes (2004+)
Var21DisplayType=Formatted
Var22Name=Derived EOD 2018 T Recode (2018+)
Var22DisplayType=Formatted
Var23Name=Derived EOD 2018 N Recode (2018+)
Var23DisplayType=Formatted
Var24Name=Derived EOD 2018 M Recode (2018+)
Var24DisplayType=Formatted
Var25Name=Derived EOD 2018 Stage Group Recode (2018+)
Var25DisplayType=Formatted
Var26Name=Derived AJCC Stage Group, 7th ed (2010-2015)
Var26DisplayType=Formatted
Var27Name=Derived AJCC T, 7th ed (2010-2015)
Var27DisplayType=Formatted
Var28Name=Derived AJCC N, 7th ed (2010-2015)
Var28DisplayType=Formatted
Var29Name=Derived AJCC M, 7th ed (2010-2015)
Var29DisplayType=Formatted
Var30Name=7th Edition Stage Group Recode (2016-2017)
Var30DisplayType=Formatted
Var31Name=Derived SEER Cmb Stg Grp (2016-2017)
Var31DisplayType=Formatted
Var32Name=Derived SEER Combined T (2016-2017)
Var32DisplayType=Formatted
Var33Name=Derived SEER Combined N (2016-2017)
Var33DisplayType=Formatted
Var34Name=Derived SEER Combined M (2016-2017)
Var34DisplayType=Formatted
Var35Name=Derived SEER Combined T Src (2016-2017)
Var35DisplayType=Formatted
Var36Name=Derived SEER Combined N Src (2016-2017)
Var36DisplayType=Formatted
Var37Name=Derived SEER Combined M Src (2016-2017)
Var37DisplayType=Formatted
Var38Name=TNM Edition Number (2016-2017)
Var38DisplayType=Formatted
Var39Name=RX Summ--Surg Prim Site (1998+)
Var39DisplayType=Formatted
Var40Name=RX Summ--Scope Reg LN Sur (2003+)
Var40DisplayType=Formatted
Var41Name=RX Summ--Surg Oth Reg/Dis (2003+)
Var41DisplayType=Formatted
Var42Name=RX Summ--Surg/Rad Seq
Var42DisplayType=Formatted
Var43Name=Reason no cancer-directed surgery
Var43DisplayType=Formatted
Var44Name=Radiation recode
Var44DisplayType=Formatted
Var45Name=Chemotherapy recode (yes, no/unk)
Var45DisplayType=Formatted
Var46Name=Scope of reg lymph nd surg (1998-2002)
Var46DisplayType=Formatted
Var47Name=RX Summ--Reg LN Examined (1998-2002)
Var47DisplayType=Formatted
Var48Name=Surgery of oth reg/dis sites (1998-2002)
Var48DisplayType=Formatted
Var49Name=Site specific surgery (1973-1997 varying detail by year and site)
Var49DisplayType=Formatted
Var50Name=Radiation to Brain or CNS Recode (1988-1997)
Var50DisplayType=Formatted
Var51Name=RX Summ--Systemic/Sur Seq (2007+)
Var51DisplayType=Formatted
Var52Name=Time from diagnosis to treatment in days recode
Var52DisplayType=Formatted
Var53Name=COD to site recode
Var53DisplayType=Formatted
Var54Name=SEER cause-specific death classification
Var54DisplayType=Formatted
Var55Name=SEER other cause of death classification
Var55DisplayType=Formatted
Var56Name=Survival months
Var56DisplayType=Formatted
Var57Name=Survival months flag
Var57DisplayType=Formatted
Var58Name=COD to site rec KM
Var58DisplayType=Formatted
Var59Name=COD to site recode ICD-O-3 2023 Revision
Var59DisplayType=Formatted
Var60Name=COD to site recode ICD-O-3 2023 Revision Expanded (1999+)
Var60DisplayType=Formatted
Var61Name=Vital status recode (study cutoff used)
Var61DisplayType=Formatted
'''

#split text into lines
lines = dictionary_text.strip().splitlines()

#Filter out lines containing 'Formatted'
name_lines = [line for line in lines if 'Formatted' not in line]

#clean up name lines by removing the 'VARxxDisplayType=' section of each line
col_names = [line.split('=', 1)[1] for line in name_lines]

In [7]:
#add column names to dataframe
seer_data = seer_data.toDF(*col_names)

In [8]:
#display CNS labeling to identify value for non-CNS related cancers
seer_data.groupBy('SEER Brain and CNS Recode').count().show(truncate = False)

+----------------------------------------------------+-------+
|SEER Brain and CNS Recode                           |count  |
+----------------------------------------------------+-------+
|1.4 Choroid plexus tumors                           |33     |
|1.1.7 Astroblastoma                                 |9      |
|1.1.1 Diffuse astrocytoma and anaplastic astrocytoma|3249   |
|1.1.6 Other astrocytic tumors                       |1397   |
|1.3 Meningiomas                                     |446    |
|1.1.10 Other                                        |47     |
|1.1.5 Oligoastrocytoma                              |15     |
|1.1.9 Glioma, unspecified                           |1922   |
|5. All Other Cancer Types                           |2237285|
|1.1.8 Ependymal tumors                              |920    |
|1.6 Other Malignant Brain/ONS                       |2077   |
|3. Malignant tumors of the pineal region            |116    |
|1.1.4 Oligodendroglioma                             |1

In [9]:
#count of missing data / CNS cancers
seer_data[
    (seer_data['Median household income inflation adj to 2023'] == 'Unknown/missing/no match/Not 1990-2023') | 
    (seer_data['Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)'] == 'Non-Hispanic Unknown Race') |
    (seer_data['Rural-Urban Continuum Code'] == 'Unknown/missing/no match (Alaska or Hawaii - Entire State)') |
    (seer_data['Rural-Urban Continuum Code'] == 'Unknown/missing/no match/Not 1990-2023') |
    (seer_data['Time from diagnosis to treatment in days recode'] == 'Unable to calculate') |
    (seer_data['Time from diagnosis to treatment in days recode'] == '731+ days') |
    (seer_data['SEER Brain and CNS Recode'] != '5. All Other Cancer Types')
].count()

570952

#### Remove Missing Data / CNS cancers

In [10]:
#remove all missing data and all CNS-related cancers
seer_data_cleaned = seer_data.filter(
    (seer_data['Median household income inflation adj to 2023'] != 'Unknown/missing/no match/Not 1990-2023') & 
    (seer_data['Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)'] != 'Non-Hispanic Unknown Race') &
    (seer_data['Rural-Urban Continuum Code'] != 'Unknown/missing/no match (Alaska or Hawaii - Entire State)') &
    (seer_data['Rural-Urban Continuum Code'] != 'Unknown/missing/no match/Not 1990-2023') &
    (seer_data['Time from diagnosis to treatment in days recode'] != 'Unable to calculate') &
    (seer_data['Time from diagnosis to treatment in days recode'] != '731+ days') &
    (seer_data['SEER Brain and CNS Recode'] == '5. All Other Cancer Types')
    )

## Descriptive Analysis

#### Total Count

In [11]:
#total dataset count prior to removal of missing values and CNS cancers
seer_data.count()

2266903

In [12]:
#total count of dataset following removal of missing values / CNS cancers
n = seer_data_cleaned.count()

print(f'Total Row Count: {n}')

Total Row Count: 1695951


## Statistical Analysis - Time to Treatment

In [13]:
#take a sample of the dataset for analysis and convert to pandas dataframe
sample_df = seer_data_cleaned.sample(fraction=0.02, seed=42)
sample_pd = sample_df.toPandas()

In [14]:
#convert numeric columns to integer data types
sample_pd['Time from diagnosis to treatment in days recode'] = sample_pd['Time from diagnosis to treatment in days recode'].astype(int)
sample_pd['Year of diagnosis'] = sample_pd['Year of diagnosis'].astype(int)
sample_pd['Survival months'] = sample_pd['Survival months'].astype(int)
sample_pd['Total number of in situ/malignant tumors for patient'] = sample_pd['Total number of in situ/malignant tumors for patient'].astype(int)
sample_pd['Total number of benign/borderline tumors for patient'] = sample_pd['Total number of benign/borderline tumors for patient'].astype(int)

In [15]:
#re-code age to larger groups that represent each decade

age_mapping = {
    '00 years': '00-19 years', 
    '01-04 years': '00-19 years',
    '05-09 years': '00-19 years',
    '10-14 years': '00-19 years',
    '15-19 years': '00-19 years',
    '20-24 years': '20-29 years',
    '25-29 years': '20-29 years',
    '30-34 years': '30-39 years',
    '35-39 years': '30-39 years',
    '40-44 years': '40-49 years',
    '45-49 years': '40-49 years',
    '50-54 years': '50-59 years',
    '55-59 years': '50-59 years',
    '60-64 years': '60-69 years',
    '65-69 years': '60-69 years',
    '70-74 years': '70-79 years',
    '75-79 years': '70-79 years',
    '80-84 years': '80-89 years',
    '85-89 years': '80-89 years',
    '90+ years': '90+ years'
}

sample_pd['Age Group'] = sample_pd['Age recode with <1 year olds and 90+'].map(age_mapping)

In [16]:
#recode income to smaller groups
income_mapping = {
    '< $40,000': '< $40,000',
    '$40,000 - $44,999': '$40,000 - $54,999',
    '$45,000 - $49,999': '$40,000 - $54,999',
    '$50,000 - $54,999': '$40,000 - $54,999',
    '$55,000 - $59,999': '$55,000 - 79,999',
    '$60,000 - $64,999': '$55,000 - 79,999',
    '$65,000 - $69,999': '$55,000 - 79,999',
    '$70,000 - $74,999': '$55,000 - 79,999',
    '$75,000 - $79,999': '$55,000 - 79,999',
    '$80,000 - $84,999': '$80,000+',
    '$85,000 - $89,999': '$80,000+',
    '$90,000 - $94,999': '$80,000+',
    '$95,000 - $99,999': '$80,000+',
    '$100,000 - $109,999': '$80,000+',
    '$110,000 - $119,999': '$80,000+',
    '$120,000+': '$80,000+'
}

sample_pd['Income Group'] = sample_pd['Median household income inflation adj to 2023'].map(income_mapping)

In [17]:
#recode year to pre-covid,covid, and post-covid groups
year_mapping = {
    2018: '2018/2019',
    2019: '2018/2019',
    2020: '2020/2021',
    2021: '2020/2021',
    2022: '2022'
}

sample_pd['Year Group'] = sample_pd['Year of diagnosis'].map(year_mapping)

In [18]:
#add binary classification column to dataset; 0 for <28 days and 1 for >=28days for 'time from diagnosis to treatment in days recode' column
sample_pd['Treatment Delay'] = sample_pd['Time from diagnosis to treatment in days recode'].apply(lambda x: 0 if x < 28 else 1)

#### Sex

In [19]:
#calculate proportions for sexes in dataset
sample_pd['Sex'].value_counts(normalize = True)

Sex
Female    0.525109
Male      0.474891
Name: proportion, dtype: float64

In [20]:
#summarize central tendency stats and quartiles for time to treatment for each sex
print('Average Time from Diagnosis to Treatment by Sex')
sample_pd.groupby('Sex')['Time from diagnosis to treatment in days recode'].describe()

Average Time from Diagnosis to Treatment by Sex


,count,mean,std,min,25%,50%,75%,max
Sex,,,,,,,,
Female,18017.0,35.371760,39.901736,0.0,5.0,29.0,49.0,715.0
Male,16294.0,43.549834,58.825752,0.0,0.0,28.0,61.0,730.0


In [21]:
#run t-test comparing average time from diagnosis to treatment in men vs. women
group1 = sample_pd[sample_pd['Sex'] == 'Male']['Time from diagnosis to treatment in days recode']
group2 = sample_pd[sample_pd['Sex'] == 'Female']['Time from diagnosis to treatment in days recode']

t_stat, p_val = stats.ttest_ind(group1, group2, equal_var = False)

print(f'T-statistic: {t_stat}, P-value: {p_val}')
print(stats.ttest_ind(group1, group2, equal_var = False).confidence_interval())

T-statistic: 14.912519613129076, P-value: 4.239640668828899e-50
ConfidenceInterval(low=7.103177607960114, high=9.252970972824293)


#### Age

In [22]:
#calculate proportions for age groups in the dataset
sample_pd['Age Group'].value_counts(normalize = True).round(2)

Age Group
60-69 years    0.30
70-79 years    0.26
50-59 years    0.18
80-89 years    0.11
40-49 years    0.08
30-39 years    0.04
90+ years      0.02
20-29 years    0.02
00-19 years    0.01
Name: proportion, dtype: float64

In [23]:
#summarize central tendency stats and quartiles for time to treatment for each recoded age group
print('Average Time from Diagnosis to Treatment by Age Group')
sample_pd.groupby('Age Group')['Time from diagnosis to treatment in days recode'].describe()

Average Time from Diagnosis to Treatment by Age Group


,count,mean,std,min,25%,50%,75%,max
Age Group,,,,,,,,
00-19 years,318.0,12.701258,34.490003,0.0,0.0,3.0,11.0,375.0
20-29 years,523.0,23.892925,38.599178,0.0,0.0,8.0,34.0,361.0
30-39 years,1272.0,27.988208,41.232133,0.0,0.0,17.0,42.0,433.0
40-49 years,2822.0,33.514174,39.831197,0.0,0.0,26.0,49.0,420.0
50-59 years,6035.0,40.843745,52.845366,0.0,2.0,29.0,55.0,730.0
60-69 years,10229.0,44.112230,54.511696,0.0,6.0,32.0,59.0,697.0
70-79 years,8921.0,40.482906,49.013807,0.0,4.0,29.0,55.0,718.0
80-89 years,3652.0,33.893209,43.383357,0.0,0.0,25.0,48.0,640.0
90+ years,539.0,32.539889,49.176677,0.0,0.0,19.0,49.0,636.0


#### Race/Ethnicity

In [24]:
#calculate proportions for race/ethnicities in dataset
sample_pd['Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)'].value_counts(normalize = True).round(2)

Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)
Non-Hispanic White                            0.68
Hispanic (All Races)                          0.13
Non-Hispanic Black                            0.10
Non-Hispanic Asian or Pacific Islander        0.08
Non-Hispanic American Indian/Alaska Native    0.01
Name: proportion, dtype: float64

In [25]:
#summarize central tendency stats and quartiles for time to treatment for each age group
print('Average Time from Diagnosis to Treatment by Age Group')
sample_pd.groupby('Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)')['Time from diagnosis to treatment in days recode'].describe().round(2)

Average Time from Diagnosis to Treatment by Age Group


,count,mean,std,min,25%,50%,75%,max
"Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)",,,,,,,,
Hispanic (All Races),4345.0,42.66,53.78,0.0,2.0,31.0,59.0,659.0
Non-Hispanic American Indian/Alaska Native,175.0,36.77,43.84,0.0,3.5,29.0,49.0,356.0
Non-Hispanic Asian or Pacific Islander,2866.0,40.32,45.43,0.0,7.0,31.0,56.0,534.0
Non-Hispanic Black,3587.0,49.75,63.83,0.0,5.0,34.0,66.0,697.0
Non-Hispanic White,23338.0,36.90,47.04,0.0,0.0,27.0,51.0,730.0


#### Income

In [26]:
#calculate proportions of income groups in the dataset
sample_pd['Income Group'].value_counts(normalize = True).round(2)

Income Group
$80,000+             0.66
$55,000 - 79,999     0.27
$40,000 - $54,999    0.06
< $40,000            0.01
Name: proportion, dtype: float64

In [27]:
#summarize central tendency stats and quartiles for time to treatment each recoded income group
print('Average Time from Diagnosis to Treatment by income Group')
sample_pd.groupby('Income Group')['Time from diagnosis to treatment in days recode'].describe().round(2)

Average Time from Diagnosis to Treatment by income Group


,count,mean,std,min,25%,50%,75%,max
Income Group,,,,,,,,
"$40,000 - $54,999",2130.0,39.21,50.63,0.0,1.0,28.0,54.0,525.0
"$55,000 - 79,999",9096.0,39.25,51.10,0.0,1.0,28.0,54.0,730.0
"$80,000+",22812.0,39.33,49.47,0.0,2.0,28.0,54.0,718.0
"< $40,000",273.0,33.82,46.95,0.0,0.0,21.0,43.0,322.0


#### Metropolitan Area

In [28]:
#calculate proportions of metropolitan groups in the dataset
sample_pd['Rural-Urban Continuum Code'].value_counts(normalize = True).round(2)

Rural-Urban Continuum Code
Counties in metropolitan areas ge 1 million pop                 0.60
Counties in metropolitan areas of 250,000 to 1 million pop      0.21
Nonmetropolitan counties adjacent to a metropolitan area        0.07
Counties in metropolitan areas of lt 250 thousand pop           0.07
Nonmetropolitan counties not adjacent to a metropolitan area    0.04
Name: proportion, dtype: float64

In [29]:
#summarize central tendency stats and quartiles for time to treatment for each metropolitan area
print('Average Time from Diagnosis to Treatment by Metropolitan Area')
sample_pd.groupby('Rural-Urban Continuum Code')['Time from diagnosis to treatment in days recode'].describe().round(2)

Average Time from Diagnosis to Treatment by Metropolitan Area


,count,mean,std,min,25%,50%,75%,max
Rural-Urban Continuum Code,,,,,,,,
Counties in metropolitan areas ge 1 million pop,20744.0,40.26,50.97,0.0,2.0,29.0,55.00,718.0
"Counties in metropolitan areas of 250,000 to 1 million pop",7116.0,38.33,48.71,0.0,0.0,28.0,53.00,636.0
Counties in metropolitan areas of lt 250 thousand pop,2444.0,37.97,51.42,0.0,1.0,27.0,52.25,730.0
Nonmetropolitan counties adjacent to a metropolitan area,2507.0,36.28,45.04,0.0,1.0,27.0,51.00,715.0
Nonmetropolitan counties not adjacent to a metropolitan area,1500.0,36.80,46.68,0.0,0.0,26.0,50.00,525.0


In [30]:
#calculate central tendency / dispersion of metropolitan and non-metropolitan time to diagnosis

metro = sample_pd[(sample_pd['Rural-Urban Continuum Code'] == 'Counties in metropolitan areas ge 1 million pop') | 
                   (sample_pd['Rural-Urban Continuum Code'] == 'Counties in metropolitan areas of 250,000 to 1 million pop') |
                   (sample_pd['Rural-Urban Continuum Code'] == 'Counties in metropolitan areas of lt 250 thousand pop')
                   ]['Time from diagnosis to treatment in days recode'].describe()

non_metro = sample_pd[(sample_pd['Rural-Urban Continuum Code'] == 'Nonmetropolitan counties adjacent to a metropolitan area') | 
                   (sample_pd['Rural-Urban Continuum Code'] == 'Nonmetropolitan counties not adjacent to a metropolitan area')
                   ]['Time from diagnosis to treatment in days recode'].describe()

print(f'Metropolitan Areas: \n{metro}\n\nNon-Metropolitan Areas: \n{non_metro}')

Metropolitan Areas: 
count    30304.000000
mean        39.623383
std         50.490727
min          0.000000
25%          2.000000
50%         28.000000
75%         54.000000
max        730.000000
Name: Time from diagnosis to treatment in days recode, dtype: float64

Non-Metropolitan Areas: 
count    4007.000000
mean       36.472922
std        45.657232
min         0.000000
25%         1.000000
50%        27.000000
75%        51.000000
max       715.000000
Name: Time from diagnosis to treatment in days recode, dtype: float64


In [31]:
#run t-test comparing metropolitan and non-metropolitan groups
met_group1 = sample_pd[(sample_pd['Rural-Urban Continuum Code'] == 'Counties in metropolitan areas ge 1 million pop') | 
                   (sample_pd['Rural-Urban Continuum Code'] == 'Counties in metropolitan areas of 250,000 to 1 million pop') |
                   (sample_pd['Rural-Urban Continuum Code'] == 'Counties in metropolitan areas of lt 250 thousand pop')
                   ]['Time from diagnosis to treatment in days recode']
met_group2 = sample_pd[(sample_pd['Rural-Urban Continuum Code'] == 'Nonmetropolitan counties adjacent to a metropolitan area') | 
                   (sample_pd['Rural-Urban Continuum Code'] == 'Nonmetropolitan counties not adjacent to a metropolitan area')
                   ]['Time from diagnosis to treatment in days recode']

t_stat, p_val = stats.ttest_ind(met_group1, met_group2, equal_var=False)

print(f'T-Statistic: {t_stat}')
print(f'P-Value: {p_val}')
print(stats.ttest_ind(met_group1, met_group2, equal_var=False).confidence_interval())

T-Statistic: 4.052529864010204
P-Value: 5.138009363660408e-05
ConfidenceInterval(low=1.6264307047260433, high=4.6744906271090265)


#### Diagnosis Year

In [32]:
#calculate proportions of year groups in the dataset
sample_pd['Year of diagnosis'].value_counts(normalize = True)

Year of diagnosis
2021    0.208388
2019    0.205590
2022    0.204453
2018    0.197488
2020    0.184081
Name: proportion, dtype: float64

In [33]:
#summarize central tendency stats and quartiles for time to treatment for each year
print('Average Time from Diagnosis to Treatment by Metropolitan Area')
sample_pd.groupby('Year of diagnosis')['Time from diagnosis to treatment in days recode'].describe()

Average Time from Diagnosis to Treatment by Metropolitan Area


,count,mean,std,min,25%,50%,75%,max
Year of diagnosis,,,,,,,,
2018,6776.0,37.645956,51.468174,0.0,0.0,26.0,50.0,715.0
2019,7054.0,37.393819,50.803090,0.0,0.0,26.0,51.0,730.0
2020,6316.0,35.782457,46.042469,0.0,1.0,25.0,49.0,636.0
2021,7150.0,41.522937,50.980060,0.0,3.0,31.0,56.0,718.0
2022,7015.0,43.497933,49.576351,0.0,5.0,33.0,61.0,526.0


In [34]:
#summarize central tendency stats and quartiles for time to treatment for each peri-COVID timeframe
sample_pd.groupby('Year Group')['Time from diagnosis to treatment in days recode'].describe().round(2)

,count,mean,std,min,25%,50%,75%,max
Year Group,,,,,,,,
2018/2019,13830.0,37.52,51.13,0.0,0.0,26.0,51.0,730.0
2020/2021,13466.0,38.83,48.81,0.0,2.0,28.0,53.0,718.0
2022,7015.0,43.50,49.58,0.0,5.0,33.0,61.0,526.0


In [35]:
#calculate proportions for peri-COVID timeframes in dataset
sample_pd['Year Group'].value_counts(normalize = True).round(2)

Year Group
2018/2019    0.40
2020/2021    0.39
2022         0.20
Name: proportion, dtype: float64

In [36]:
#calculate central tendency / dispersion of pre-COVID vs. COVID time to diagnosis

pre_covid = sample_pd[(sample_pd['Year Group'] == '2018/2019')]['Time from diagnosis to treatment in days recode'].describe()

covid = sample_pd[(sample_pd['Year Group'] == '2020/2021')]['Time from diagnosis to treatment in days recode'].describe()

post_covid = sample_pd[(sample_pd['Year Group'] == '2022')]['Time from diagnosis to treatment in days recode'].describe()

print(f'Pre-COVID: \n{pre_covid}\n\nCOVID: \n{covid}\n\nPost-COVID: \n{post_covid}')

Pre-COVID: 
count    13830.000000
mean        37.517354
std         51.128334
min          0.000000
25%          0.000000
50%         26.000000
75%         51.000000
max        730.000000
Name: Time from diagnosis to treatment in days recode, dtype: float64

COVID: 
count    13466.000000
mean        38.830462
std         48.808870
min          0.000000
25%          2.000000
50%         28.000000
75%         53.000000
max        718.000000
Name: Time from diagnosis to treatment in days recode, dtype: float64

Post-COVID: 
count    7015.000000
mean       43.497933
std        49.576351
min         0.000000
25%         5.000000
50%        33.000000
75%        61.000000
max       526.000000
Name: Time from diagnosis to treatment in days recode, dtype: float64


In [37]:
#run ANOVA comparing pre-COVID vs. COVID vs. post-COVID
group0 = sample_pd[sample_pd['Year Group'] == '2018/2019']['Time from diagnosis to treatment in days recode']
group1 = sample_pd[sample_pd['Year Group'] == '2020/2021']['Time from diagnosis to treatment in days recode']
group2 = sample_pd[sample_pd['Year Group'] == '2022']['Time from diagnosis to treatment in days recode']

f_stat, p_val = stats.f_oneway(group0,
                               group1, 
                               group2, 
                               )
print(f'F-statistic: {f_stat}')
print(f'P-value: {p_val}')

F-statistic: 34.21532053716838
P-value: 1.4297983819472757e-15


In [38]:
#perform tukey hsd post-hoc analysis for pairwise comparisons
from scipy.stats import tukey_hsd

year_tukeyhsd = tukey_hsd(group0,
                         group1, 
                         group2,
                         )

print(year_tukeyhsd)

Tukey's HSD Pairwise Group Comparisons (95.0% Confidence Interval)
Comparison  Statistic  p-value  Lower CI  Upper CI
 (0 - 1)     -1.313     0.076    -2.729     0.103
 (0 - 2)     -5.981     0.000    -7.695    -4.266
 (1 - 0)      1.313     0.076    -0.103     2.729
 (1 - 2)     -4.667     0.000    -6.390    -2.945
 (2 - 0)      5.981     0.000     4.266     7.695
 (2 - 1)      4.667     0.000     2.945     6.390



In [39]:
#run chi2 test on categrocial 'Treatment Delay' column
from scipy.stats import chi2_contingency

#run chi2 of categorical 'Treatment Delay' column

#contingency table
year_ct = pd.crosstab(sample_pd['Year Group'], sample_pd['Treatment Delay'])
print('Year Group Contingency Table:')
print(year_ct)

Year Group Contingency Table:
Treatment Delay     0     1
Year Group                 
2018/2019        7125  6705
2020/2021        6588  6878
2022             3027  3988


In [40]:
pre_covid_delay_freq = 6710 / (6710 + 7172)
covid_delay_freq = 6920 / (6920 + 6876)
post_covid_dela_freq = 3964 / (3964 + 3161)

print(f'Pre-COVID Treatment Delay Frequency: {pre_covid_delay_freq}')
print(f'COVID Treatment Delay Frequency: {covid_delay_freq}')
print(f'Post-COVID Treatment Delay Frequency: {post_covid_dela_freq}')

Pre-COVID Treatment Delay Frequency: 0.48335974643423135
COVID Treatment Delay Frequency: 0.5015946651203247
Post-COVID Treatment Delay Frequency: 0.5563508771929825


In [41]:
chi2_stat, p_value, dof, expected_freq = chi2_contingency(year_ct)

print(f"\nChi-square statistic: {chi2_stat:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected_freq)


Chi-square statistic: 130.5998
P-value: 0.0000
Degrees of freedom: 2
Expected frequencies:
[[6747.52120311 7082.47879689]
 [6569.92917723 6896.07082277]
 [3422.54961966 3592.45038034]]


In [42]:
#post-hoc analysis
from itertools import combinations
from scipy.stats import chi2_contingency
from statsmodels.stats.multitest import multipletests

#contingency table
year_ct = pd.crosstab(sample_pd['Year Group'], sample_pd['Treatment Delay'])

# Store pairwise p-values and labels
pairwise_pvals = []
pair_labels = []

# Get unique Year Groups
year_groups = year_ct.index.tolist()

# Loop through all unique pairs
for g1, g2 in combinations(year_groups, 2):
    sub_table = year_ct.loc[[g1, g2]]  # Subset of 2 rows
    chi2, p, _, _ = chi2_contingency(sub_table)
    pairwise_pvals.append(p)
    pair_labels.append(f"{g1} vs {g2}")

# Bonferroni correction for multiple tests
reject, pvals_corrected, _, _ = multipletests(pairwise_pvals, method='bonferroni')

# Display results
print("\nPost-hoc pairwise chi2 tests with Bonferroni correction:")
for label, raw_p, corr_p, rej in zip(pair_labels, pairwise_pvals, pvals_corrected, reject):
    print(f"{label}: raw p = {raw_p:.4f}, corrected p = {corr_p:.4f}, reject null = {rej}")



Post-hoc pairwise chi2 tests with Bonferroni correction:
2018/2019 vs 2020/2021: raw p = 0.0000, corrected p = 0.0001, reject null = True
2018/2019 vs 2022: raw p = 0.0000, corrected p = 0.0000, reject null = True
2020/2021 vs 2022: raw p = 0.0000, corrected p = 0.0000, reject null = True


In [43]:
#run t-test comparing pre-COVID vs. COVID diagnosis years
group1 = sample_pd[(sample_pd['Year of diagnosis'] == '2018') | 
                   (sample_pd['Year of diagnosis'] == '2019')
                   ]['Time from diagnosis to treatment in days recode']
group2 = sample_pd[(sample_pd['Year of diagnosis'] == '2020') | 
                   (sample_pd['Year of diagnosis'] == '2021') |
                   (sample_pd['Year of diagnosis'] == '2022')
                   ]['Time from diagnosis to treatment in days recode']

t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)

print(f'T-Statistic: {t_stat}')
print(f'P-Value: {p_val}')

T-Statistic: nan
P-Value: nan


/opt/anaconda3/envs/bmi6318/lib/python3.10/site-packages/scipy/_lib/deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


### Logistic Regression - Colorectal Cancer

In [44]:
#create dataframe of only colorectal primary sites
colorectal_df = sample_pd[
    (sample_pd['Primary Site - labeled'] == 'C18.0-Cecum') |
    (sample_pd['Primary Site - labeled'] == 'C18.1-Appendix') |
    (sample_pd['Primary Site - labeled'] == 'C18.2-Ascending colon') |
    (sample_pd['Primary Site - labeled'] == 'C18.3-Hepatic flexure of colon') |
    (sample_pd['Primary Site - labeled'] == 'C18.4-Transverse colon') |
    (sample_pd['Primary Site - labeled'] == 'C18.5-Splenic flexure of colon') |
    (sample_pd['Primary Site - labeled'] == 'C18.6-Descending colon') |
    (sample_pd['Primary Site - labeled'] == 'C18.7-Sigmoid colon') |
    (sample_pd['Primary Site - labeled'] == 'C18.8-Overlapping lesion of colon') |
    (sample_pd['Primary Site - labeled'] == 'C18.9-Colon, NOS') |
    (sample_pd['Primary Site - labeled'] == 'C19.9-Rectosigmoid junction') |
    (sample_pd['Primary Site - labeled'] == 'C20.9-Rectum, NOS')
]


In [45]:
#check colorectal cancer filtering
colorectal_df['Primary Site - labeled'].value_counts().sort_values()

Primary Site - labeled
C18.8-Overlapping lesion of colon     24
C18.9-Colon, NOS                      44
C18.5-Splenic flexure of colon        61
C18.3-Hepatic flexure of colon        84
C18.6-Descending colon               120
C18.1-Appendix                       145
C18.4-Transverse colon               193
C19.9-Rectosigmoid junction          204
C18.0-Cecum                          435
C18.2-Ascending colon                437
C18.7-Sigmoid colon                  606
C20.9-Rectum, NOS                    781
Name: count, dtype: int64

In [46]:
#filter for  columns of interest
logreg_df = colorectal_df[[
    'Sex',
    'Age Group',
    'Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)',
    'Income Group',
    'Rural-Urban Continuum Code',
    'Year Group',
    'Time from diagnosis to treatment in days recode'
]]

In [47]:
#add binary classification column to dataset; 0 for <28 days and 1 for >=28days for 'time from diagnosis to treatment in days recode' column
logreg_df['Treatment Delay'] = logreg_df['Time from diagnosis to treatment in days recode'].apply(lambda x: 0 if x < 28 else 1)

/var/folders/db/00b5nl595fldz7p9j42cw4q80000gn/T/ipykernel_48349/3267392979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logreg_df['Treatment Delay'] = logreg_df['Time from diagnosis to treatment in days recode'].apply(lambda x: 0 if x < 28 else 1)


In [48]:
#drop age group 00-19 years since all patients in this group had a time to treatment of 0 days, resulting in skewed logistic regression results
logreg_df = logreg_df[logreg_df['Age Group'] != "00-19 years"]

In [49]:
#encode categorical variables
categorical_cols = [
    'Sex',
    'Age Group',
    'Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)',
    'Income Group',
    'Rural-Urban Continuum Code',
    'Year Group'
]

logreg_df_categorical = pd.get_dummies(logreg_df[categorical_cols], drop_first = False).astype(int)

logreg_df_numeric = logreg_df[['Treatment Delay']]

logreg_df_encoded = pd.concat([logreg_df_categorical, logreg_df_numeric], axis = 1)

In [50]:
#manually drop reference groups
logreg_df_final = logreg_df_encoded.drop(columns = [
    'Sex_Female',
    'Age Group_50-59 years',
    'Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)_Non-Hispanic White',
    'Income Group_$80,000+',
    'Rural-Urban Continuum Code_Counties in metropolitan areas ge 1 million pop',
    'Year Group_2018/2019'
])

In [ ]:
import statsmodels.api as sm

#run logistic regression
X = logreg_df_final.drop(columns = 'Treatment Delay')
Y = logreg_df_final['Treatment Delay']

log_reg = sm.Logit(Y, X).fit()

print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.657923
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:        Treatment Delay   No. Observations:                 3118
Model:                          Logit   Df Residuals:                     3097
Method:                           MLE   Df Model:                           20
Date:                Thu, 07 Aug 2025   Pseudo R-squ.:                0.002427
Time:                        16:16:33   Log-Likelihood:                -2051.4
converged:                       True   LL-Null:                       -2056.4
Covariance Type:            nonrobust   LLR p-value:                    0.9685
                                                                                                            coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [52]:
#save output as a csv for easy input into report
log_reg_csv = log_reg.summary().as_csv()

log_reg_filename = 'logistic_regression.csv'

with open(log_reg_filename, 'w') as f:
    f.write(log_reg_csv)

### Multiple Linear Regression - Colorectal Cancer

In [53]:
#encode categorical variables
categorical_cols = [
    'Sex',
    'Age Group',
    'Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)',
    'Income Group',
    'Rural-Urban Continuum Code',
    'Year Group'
]

linreg_df_categorical = pd.get_dummies(sample_pd[categorical_cols], drop_first = False).astype(int)

linreg_df_numeric = sample_pd['Time from diagnosis to treatment in days recode']

linreg_df_encoded = pd.concat([linreg_df_categorical, linreg_df_numeric], axis = 1)

In [54]:
#manually drop reference groups
linreg_df_final = linreg_df_encoded.drop(columns = [
    'Sex_Female',
    'Age Group_00-19 years',
    'Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)_Non-Hispanic White',
    'Income Group_$80,000+',
    'Rural-Urban Continuum Code_Counties in metropolitan areas ge 1 million pop',
    'Year Group_2018/2019'
])

In [ ]:
#run linear regression
X = linreg_df_final.drop(columns = 'Time from diagnosis to treatment in days recode')
Y = linreg_df_final['Time from diagnosis to treatment in days recode']

X = sm.add_constant(X)

lr_model = sm.OLS(Y, X).fit()

print(lr_model.summary())


                                           OLS Regression Results                                          
Dep. Variable:     Time from diagnosis to treatment in days recode   R-squared:                       0.027
Model:                                                         OLS   Adj. R-squared:                  0.027
Method:                                              Least Squares   F-statistic:                     43.88
Date:                                             Thu, 07 Aug 2025   Prob (F-statistic):          2.67e-187
Time:                                                     16:16:34   Log-Likelihood:            -1.8241e+05
No. Observations:                                            34311   AIC:                         3.649e+05
Df Residuals:                                                34288   BIC:                         3.651e+05
Df Model:                                                       22                                         
Covariance Type:            

In [56]:
#save output as a csv for easy input into report
lin_reg_csv = lr_model.summary().as_csv()

lin_reg_filename = 'linear_regression.csv'

with open(lin_reg_filename, 'w') as f:
    f.write(lin_reg_csv)